# Plotting: 1D and 2D data

This notebook show how to visualize 1D and 2d simulation data.

In [ ]:
import osyris
import numpy as np
import matplotlib.pyplot as plt

## Plotting 1D data

1D and 2D data are loaded in exactly the same way as 3D simulations,
by creating a `Dataset` and calling the `load()` method.

In [ ]:
path = "osyrisdata/sod"
data = osyris.Dataset(2, scale="cm", path=path).load()
data

### Line profile

To plot a simple line profile of density as a function of position,
Osyris provides the `plot` function which is analogous to Matplotlib's
[plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html) function.

In [ ]:
osyris.plot(data['amr']['xyz_x'], data['hydro']['density'])

### Styling lines

Because the function is based on Matplotlib's implementation,
it also supports Matplotlib's styling arguments.
For instance, to plot using red marker instead of a solid line,
you can do

In [ ]:
osyris.plot(data['amr']['xyz_x'], data['hydro']['density'],
           marker='o', ls='None', color='red')

### Logarithmic scales

Osyris does provide some convenience arguments in addition to Matplotlib's functionalty.
For example, to set a logarithmic scale on the x and y axes, use the `logx` and `logy` arguments

In [ ]:
osyris.plot(data['amr']['xyz_x'], data['hydro']['density'],
           marker='o', ls='None', logx=True, logy=True)

### Multiple lines

It is also possible to over-plot multiple lines in one go,
by supplying more than one array for the `y` values.

However, note that to be able to plot the variables on a single axis,
they must all have the same unit.

In [ ]:
try:
    osyris.plot(data['amr']['xyz_x'],
                data['hydro']['density'],
                data['hydro']['pressure'])
except Exception as e:
    print(e)

To be able to show an example, we fake a new density variable.

In [ ]:
data['hydro']['new_density'] = data['hydro']['density'] * 2
osyris.plot(data['amr']['xyz_x'],
            data['hydro']['density'],
            data['hydro']['new_density'])

By default, the same x coordinate will be used for both sets of y values.
It is however possible to plot on the same axes two variables with different x coordinates.

In this case, the two entries must be dicts, and they must contain at least the entries `'x'` and `'y'`.
This is useful in the case where one wishes to compare two outputs from different times,
which do not contain the same number of mesh cells (and hence cannot have the same x coordinate).

In [ ]:
old = osyris.Dataset(1, scale="cm", path=path).load()

osyris.plot({'x': data['amr']['xyz_x'], 'y': data['hydro']['density']},
            {'x': old['amr']['xyz_x'], 'y': old['hydro']['density']},
            marker='o', ls='None')

## Subplots / tiled plots

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 9))

osyris.plot(data['amr']['xyz_x'], data['hydro']['density'],
            marker='o', ls='None', ax=ax[0, 0])
osyris.plot(data['amr']['xyz_x'], data['hydro']['velocity_x'],
            color='green', ax=ax[0, 1])
osyris.plot(data['amr']['xyz_x'], data['hydro']['pressure'],
            color='red', ax=ax[1, 0])
osyris.plot(data['amr']['xyz_x'], data['amr']['level'],
            color='black', ls='dotted', ax=ax[1, 1])

## Plotting 2D data

In [ ]:
path = "osyrisdata/sod"
data2d = osyris.Dataset(2, scale="cm").load()

In [ ]:
osyris.map({"data": data2d['hydro']["density"], "norm": "log"}, # layer 1
             # dx=2,
             # origin=osyris.Array([0.5, 0.5], unit='cm'),
             cmap='jet',
             # resolution=512,
             # resolution={'x': 97, 'y': 97}
            )

In [ ]:
osyris.map({"data": data2d['amr']["level"]}, # layer 1
#              dx=2,
#              origin=osyris.Array([0.5, 0.5], unit='cm'),
             cmap='jet',
             # resolution={'x': 97, 'y': 97}
            )